In [ ]:
import blinpy as bp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
from scipy.optimize import minimize
%matplotlib inline

## Figure 2: various orders for difference priors

In [ ]:
# generate random data
xobs = -1.75 + 3.5*np.random.random(100)
yobs = 3*xobs**4-6*xobs**2+2 + np.random.randn(len(xobs))

data = pd.DataFrame({'x': xobs, 'y': yobs})

In [ ]:
xfit = np.linspace(-2,2,20)
nfit = len(xfit)

# function for generating the GAM specification given prior order and variance
generate_gam_spec = lambda pri_order, pri_var: [
    {
        'fun': lambda df: bp.utils.interp_matrix(df['x'].values, xfit, sparse=False),
        'name': 'smoothfun',
        'prior': {
            'B': bp.utils.diffmat(nfit, order=pri_order, sparse=False),
            'mu': np.zeros(nfit-pri_order),
            'cov': pri_var*np.ones(nfit-pri_order)
        }
    }
]

In [ ]:
pri_covs = [100, 1, 0.1, 0.000000001]

# fit the models with various prior orders and variances and collect the posterior means
yfits1 = [bp.models.GamModel('y', generate_gam_spec(1, pri_cov)).fit(data).post_mu for pri_cov in pri_covs]
yfits2 = [bp.models.GamModel('y', generate_gam_spec(2, pri_cov)).fit(data).post_mu for pri_cov in pri_covs]
yfits3 = [bp.models.GamModel('y', generate_gam_spec(3, pri_cov)).fit(data).post_mu for pri_cov in pri_covs]

In [ ]:
plt.figure(figsize=(9,3))
plt.subplot(131)
plt.plot(xobs, yobs, 'k.')
plt.plot(xfit, yfits1[0], '-', lw=1)
plt.plot(xfit, yfits1[1], '-')
plt.plot(xfit, yfits1[2], '-')
plt.plot(xfit, yfits1[3], '-')
plt.ylim((-4,20))
plt.xlim((-2.1,2.1))
plt.legend(['data','$\sigma=100$','$\sigma=1$','$\sigma=0.1$','$\sigma=0$'], fontsize=8)
plt.title('1st order diff')
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)

plt.subplot(132)
plt.plot(xobs, yobs, 'k.')
plt.plot(xfit, yfits2[0], '-')
plt.plot(xfit, yfits2[1], '-')
plt.plot(xfit, yfits2[2], '-')
plt.plot(xfit, yfits2[3], '-')
plt.ylim((-4,20))
plt.xlim((-2.1,2.1))
plt.xlabel('x')
plt.title('2nd order diff')
plt.grid(True)

plt.subplot(133)
plt.plot(xobs, yobs, 'k.')
plt.plot(xfit, yfits3[0], '-')
plt.plot(xfit, yfits3[1], '-')
plt.plot(xfit, yfits3[2], '-')
plt.plot(xfit, yfits3[3], '-')
plt.title('3rd order diff')
plt.ylim((-4,20))
plt.xlim((-2.1,2.1))
plt.xlabel('x')
plt.grid(True)

plt.tight_layout()
plt.show()

## Figure 8: MAP for hyperparameters

In [ ]:
pri_covs = np.logspace(-2, 3, base=10)

# calculate the likelihoods with various prior orders and variances
L1 = [bp.models.GamModel('y', generate_gam_spec(1, pri_cov**2)).fit(data).L for pri_cov in pri_covs]
L2 = [bp.models.GamModel('y', generate_gam_spec(2, pri_cov**2)).fit(data).L for pri_cov in pri_covs]
L3 = [bp.models.GamModel('y', generate_gam_spec(3, pri_cov**2)).fit(data).L for pri_cov in pri_covs]

In [ ]:
plt.figure(figsize=(6,4))
plt.semilogx(pri_covs, L1, 'k-')
plt.semilogx(pri_covs, L2, 'r-')
plt.semilogx(pri_covs, L3, 'b-')

plt.xlabel('prior standard deviation')
plt.ylabel('-2*log(posterior)')
plt.legend(['order=1', 'order=2', 'order=3'])
plt.grid(True)
plt.show()

## Figure 9: comparison of methods for hyperparameter estimation

In [ ]:
pri_covs = np.logspace(-3, 2, base=10)

# form separate test data for posterior predictive score calculation
xtest = -1.75 + 3.5*np.random.random(10)
ytest = 3*xtest**4-6*xtest**2+2 + np.random.randn(len(xtest))
test_data = pd.DataFrame({'x': xtest, 'y': ytest})

# calculate likelihoods and posterior predictive scores
L2_map = [bp.models.GamModel('y', generate_gam_spec(2, pri_cov**2)).fit(data).L for pri_cov in pri_covs]
L2_post = [bp.models.GamModel('y', generate_gam_spec(2, pri_cov**2)).fit(data).postpred(test_data) for pri_cov in pri_covs]

In [ ]:
plt.figure(figsize=(6,4))
plt.subplot(211)
plt.semilogx(pri_covs, L2_map, 'r-')
plt.grid(True)
plt.axvline(pri_covs[np.argmin(L2_map)], color='0.5')
plt.ylabel('-2*log(posterior)')

plt.subplot(212)
plt.semilogx(pri_covs, L2_post, 'b-')
plt.axvline(pri_covs[np.argmin(L2_post)], color='0.5')
plt.xlabel('prior standard deviation')
plt.ylabel('post. pred. score')
plt.grid(True)

plt.tight_layout()
plt.show()

## Figure 3: different prior order for different dimensions

In [ ]:
# generate random data
x1 = 2*np.random.random(200)
x2 = np.random.random(200)
yobs = 0.5*x1 + 4*(x2-0.5)**2/(1 + 2*x1)

data = pd.DataFrame({'x1': x1, 'x2': x2, 'y': yobs})

In [ ]:
# define the grid points for function estimation
grids = (np.linspace(-0.25,2.25,25), np.linspace(-0.25,1.25,15))
shape = [len(x) for x in grids]

# diff priors along different dimensions 
D1 = bp.utils.diffmatn(shape, 0, order=2)
D2 = bp.utils.diffmatn(shape, 1, order=3)
D = sparse.vstack((D1,D2))

# prior variances
pri_var = np.concatenate((0.001*np.ones(D1.shape[0]), 0.01*np.ones(D2.shape[0])))

# GAM specification for 2d fit
gam_spec = [
    {
        'fun': lambda df: bp.utils.interpn_matrix(df[['x1','x2']].values, grids),
        'name': 'smoothfun_2d',
        'prior': {
            'B': D,
            'mu': np.zeros(D.shape[0]),
            'cov': pri_var
        }
    }
]

In [ ]:
# fit the model
yfit = bp.models.GamModel('y', gam_spec).fit(data, obs_cov=0.01**2).post_mu

In [ ]:
from mpl_toolkits.mplot3d import axes3d

X, Y = np.meshgrid(*grids, indexing='ij')

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')

ax.plot_wireframe(X, Y, yfit.reshape(shape), alpha=0.5)
ax.plot(x1, x2, yobs, 'k.', ms=5)
plt.xlim((-0.5,2.5))
plt.ylim((-0.5,1.5))
plt.xlabel('x1')
plt.ylabel('x2')
ax.view_init(elev=14., azim=-170)
plt.show()

## Figure 4: spatially varying smoothness

In [ ]:
# generate random data
xobs = 3*np.random.random(400)
yobs = np.sin(xobs**3)+ 0.1*np.random.randn(len(xobs))

data = pd.DataFrame({'x': xobs, 'y': yobs})

In [ ]:
xfit = np.linspace(0,3,100)
nfit = len(xfit)

# function for generating GAM specification given prior variance
generate_gam_spec = lambda pri_var: [
    {
        'fun': lambda df: bp.utils.interp_matrix(df['x'].values, xfit, sparse=False),
        'name': 'smoothfun',
        'prior': {
            'B': bp.utils.diffmat(nfit, order=2, sparse=False),
            'mu': np.zeros(nfit-2),
            'cov': pri_var
        }
    }
]

In [ ]:
# choose a spatially varying covariance manually
pri_cov = (10**(-4 + 4./3*xfit))[1:-1]

# fit the models with various prior variances
yfit1 = bp.models.GamModel('y', generate_gam_spec(0.1*np.ones(nfit-2))).fit(data, obs_cov=0.1**2).post_mu
yfit2 = bp.models.GamModel('y', generate_gam_spec(0.001*np.ones(nfit-2))).fit(data, obs_cov=0.1**2).post_mu
yfit3 = bp.models.GamModel('y', generate_gam_spec(pri_cov)).fit(data, obs_cov=0.1**2).post_mu

In [ ]:
plt.figure(figsize=(9,3))

plt.subplot(131)
plt.plot(xobs, yobs, 'k.')
plt.plot(xfit, yfit1, 'r-')
plt.xlim(0,3)
plt.xlabel('x')
plt.grid(True)
plt.title('$\sigma=0.1$')

plt.subplot(132)
plt.plot(xobs, yobs, 'k.')
plt.plot(xfit, yfit2, 'r-')
plt.xlim(0,3)
plt.xlabel('x')
plt.grid(True)
plt.title('$\sigma=0.001$')

plt.subplot(133)
plt.plot(xobs, yobs, 'k.')
plt.plot(xfit, yfit3, 'r-')
plt.xlim(0,3)
plt.xlabel('x')
plt.grid(True)
plt.title('$\log_{10}\sigma=-4+(4/3)x$')

plt.tight_layout()
plt.show()

## Figure 5: sudden jump

In [ ]:
# generate data
xobs = 3*np.random.random(100)
yobs = np.sin(xobs) + (xobs > 2)*xobs + 0.1*np.random.randn(len(xobs))

data = pd.DataFrame({'x': xobs, 'y': yobs})

In [ ]:
xfit = np.linspace(0,3,50)
n = len(xfit)

# function for generating GAM specs given prior variance
generate_gam_spec = lambda pri_var: [
    {
        'fun': lambda df: bp.utils.interp_matrix(df['x'].values, xfit, sparse=False),
        'name': 'smoothfun',
        'prior': {
            'B': bp.utils.diffmat(n, order=1, sparse=False),
            'mu': np.zeros(n-1),
            'cov': pri_var
        }
    }
]

In [ ]:
# increase the prior variance at the discontinuity
pri_covs = 0.001*np.ones(n-1)
pri_covs[32] = 10000

# fit the models
yfit1 = bp.models.GamModel('y', generate_gam_spec(0.001*np.ones(n-1))).fit(data, obs_cov=0.1**2).post_mu
yfit2 = bp.models.GamModel('y', generate_gam_spec(pri_covs)).fit(data, obs_cov=0.1**2).post_mu

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(xobs, yobs, 'k.')
plt.plot(xfit, yfit1, 'b-')
plt.plot(xfit, yfit2, 'r-')
plt.xlabel('x')
plt.ylabel('y')
plt.legend(['data', 'constant smoothness', 'varying smoothness'])
plt.grid(True)
plt.show()

## Figure 6: symmetric and periodic function

In [ ]:
# generate data
xobs = -np.pi + 2*np.pi*np.random.random(50)
yobs = -np.sin(xobs)**3 + np.cos(xobs)**3 + 0.1*np.random.randn(len(xobs))

data = pd.DataFrame({'x': xobs, 'y': yobs})

In [ ]:
# define the fitting grid
xfit = np.linspace(-np.pi,np.pi,100)
ytrue = -np.sin(xfit)**3 + np.cos(xfit)**3
n = len(xfit)

# generate GAM spec given prior system matrix and variance
generate_gam_spec = lambda B, pri_var: [
    {
        'fun': lambda df: bp.utils.interp_matrix(df['x'].values, xfit, sparse=False),
        'name': 'smoothfun',
        'prior': {
            'B': B,
            'mu': np.zeros(B.shape[0]),
            'cov': pri_var
        }
    }
]

In [ ]:
# case 1: just smoothness prior
D_smooth = bp.utils.diffmat(n, order=2)
var_smooth = 0.01*np.ones(D_smooth.shape[0])
gam_spec_smooth = generate_gam_spec(D_smooth, var_smooth)

# case 2: periodic smoothness prior
D_periodic = bp.utils.diffmat(n, order=2, periodic=True)
var_periodic = 0.01*np.ones(D_periodic.shape[0])
gam_spec_periodic = generate_gam_spec(D_periodic, var_periodic)

# symmetric prior
D_symmetric = bp.utils.symmat(n, nsymm=np.where(xfit >= -np.pi/4)[0][0])
var_symmetric = 0.01*np.ones(D_symmetric.shape[0])

# case 3: periodic and symmetric priors combined
D_per_symm = sparse.vstack((D_periodic, D_symmetric))
var_per_symm = np.concatenate((var_periodic, var_symmetric))
gam_spec_both = generate_gam_spec(D_per_symm, var_per_symm)

In [ ]:
# fit the models
yfit_smooth = bp.models.GamModel('y', gam_spec_smooth).fit(data, obs_cov=0.1).post_mu
yfit_periodic = bp.models.GamModel('y', gam_spec_periodic).fit(data, obs_cov=0.1).post_mu
yfit_both = bp.models.GamModel('y', gam_spec_both).fit(data, obs_cov=0.1).post_mu

In [ ]:
plt.figure(figsize=(9,3))

plt.subplot(131)
plt.plot(xobs, yobs, 'k.')
plt.plot(xfit, ytrue, 'b--')
plt.plot(xfit, yfit_smooth, 'r-')
plt.grid(True)
plt.legend(['data','truth','fit'])
plt.xlabel('x')
plt.ylabel('y')
plt.title('smooth', fontsize=10)

plt.subplot(132)
plt.plot(xobs, yobs, 'k.')
plt.plot(xfit, ytrue, 'b--')
plt.plot(xfit, yfit_periodic, 'r-')
plt.grid(True)
plt.xlabel('x')
plt.title('smooth + periodic', fontsize=10)

plt.subplot(133)
plt.plot(xobs, yobs, 'k.')
plt.plot(xfit, ytrue, 'b--')
plt.plot(xfit, yfit_both, 'r-')
plt.grid(True)
plt.xlabel('x')
plt.title('smooth + periodic + symmetric', fontsize=10)

plt.tight_layout()
plt.show()

## Figure 7: monotonic function

In [ ]:
# generate data
xobs = np.random.random(50)
yobs = 0.5+0.2*xobs + 0.05*np.random.randn(len(xobs))

In [ ]:
# fit the model without monotonicity
xfit = np.linspace(0,1,10)
A = bp.utils.interp_matrix(xobs, xfit, sparse=False)
yfit, _, _ = bp.utils.linfit(yobs, A)

In [ ]:
# fit the model with monotonicity
C = -bp.utils.diffmat(len(xfit))
b = np.zeros(C.shape[0])

yfit_pos = bp.utils.linfit_con(yobs, A, C=C, b=b)

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(xobs,yobs,'k.')
plt.plot(xfit, yfit, 'r-')
plt.plot(xfit, yfit_pos, 'b-')
plt.xlabel('x')
plt.ylabel('y')
plt.legend(['data','no monotonicity', 'monotonicity'])
plt.grid(True)

plt.show()

## Figure 12: Hyperpar optimization

In [ ]:
# generate fit and test data
x1 = np.random.random(50)
x2 = np.random.random(50)
x3 = np.random.random(50)

x1test = np.random.random(20)
x2test = np.random.random(20)
x3test = np.random.random(20)

f1 = lambda x: 2*np.sin(np.pi*x)
f2 = lambda x: np.exp(2*x)
f3 = lambda x: x**11*(10*(1-x))**6/5+10**4*x**3*(1-x)**10

ysig = 0.1

y = f1(x1) + f2(x2) + f3(x3) + ysig*np.random.randn(len(x1))
ytest = f1(x1test) + f2(x2test) + f3(x3test) + ysig*np.random.randn(len(x1test))

data = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3, 'y': y})
test_data = pd.DataFrame({'x1': x1test, 'x2': x2test, 'x3': x3test, 'y': ytest})

In [ ]:
# define the fitting grid
nfit=50
x1fit = np.linspace(0, 1, nfit)
x2fit = np.linspace(0, 1, nfit)
x3fit = np.linspace(0, 1, nfit)

In [ ]:
# use the smooth_diff1 helper to define simple 1d smooth GAMs
gam_spec = lambda logv: [
    bp.utils.smooth_diff1('x1', x1fit, diff_std=np.exp(logv[0]), std=1e3, name='f1'),
    bp.utils.smooth_diff1('x2', x2fit, diff_std=np.exp(logv[1]), std=1e3, name='f2'),
    bp.utils.smooth_diff1('x3', x3fit, diff_std=np.exp(logv[2]), std=1e3, name='f3'),
]

In [ ]:
# fit the model with manually chosen hyperparameters
model = bp.models.GamModel('y', gam_spec(np.array([-5, -5, -2]))).fit(data, obs_cov=ysig**2)

In [ ]:
plt.figure()

plt.subplot(311)
plt.plot(x1fit, model.theta['f1'])
plt.plot(x1, f1(x1), '.')

plt.subplot(312)
plt.plot(x2fit, model.theta['f2'])
plt.plot(x2, f2(x2), '.')

plt.subplot(313)
plt.plot(x3fit, model.theta['f3'])
plt.plot(x3, f3(x3), '.')


plt.show()

In [ ]:
# define functions for calculating the likelihood and posterior predictive scores
like = lambda v: bp.models.GamModel('y', gam_spec(v)).fit(data, obs_cov=ysig**2).L
postpred = lambda v: bp.models.GamModel('y', gam_spec(v)).fit(data, obs_cov=ysig**2).postpred(test_data, obs_cov=ysig**2)

In [ ]:
# fit the models with optimized hyperparameters
v0 = np.array([-1,-1,-1])
model_like = bp.models.GamModel('y', gam_spec(minimize(like, v0, method='Nelder-Mead').x)).fit(data, obs_cov=ysig**2)
model_post = bp.models.GamModel('y', gam_spec(minimize(postpred, v0, method='Nelder-Mead').x)).fit(data, obs_cov=ysig**2)

In [ ]:
plt.figure(figsize=(6,6))

plt.subplot(311)
plt.plot(x1fit, model_like.theta['f1'])
plt.plot(x1fit, model_post.theta['f1'],'--')
plt.plot(x1, f1(x1), '.')
plt.legend(['MAP','post-pred','truth'], fontsize=8, loc=1)
plt.grid(True)
plt.ylabel('f1(x)')

plt.subplot(312)
plt.plot(x1fit, model_like.theta['f2'])
plt.plot(x1fit, model_post.theta['f2'],'--')
plt.plot(x2, f2(x2), '.')
plt.grid(True)
plt.ylabel('f2(x)')

plt.subplot(313)
plt.plot(x1fit, model_like.theta['f3'])
plt.plot(x1fit, model_post.theta['f3'],'--')
plt.plot(x3, f3(x3), '.')
plt.grid(True)
plt.xlabel('x')
plt.ylabel('f3(x)')

plt.tight_layout()
plt.show()

## Figures 10-11: Manua Loa

In [ ]:
# read the data from csv
data = pd.read_csv('manua_loa_co2.csv', index_col='Date', parse_dates=True, dayfirst=True)
data['time'] = np.arange(1, len(data)+1)

In [ ]:
# define fitting grid
time_fit = np.arange(0, len(data)+1)
month_fit = np.linspace(1, 12, 100)
nfit = len(month_fit)

# define the GAM specs: smooth along time, smooth and periodic across months
gam_specs =  lambda logv: [
    bp.utils.smooth_diff1('time', time_fit, diff_std=np.exp(logv[0]), mu=np.mean(data.co2.values), std=100),
    {
        'fun': lambda df: bp.utils.interp_matrix(df['month'].values, month_fit),
        'name': 'f(month)',
        'prior': {
            'B': bp.utils.diffmat(nfit, order=2, periodic=True),
            'mu': np.zeros(nfit),
            'cov': np.concatenate((np.exp(logv[1])*np.ones(nfit-2), 0.00001*np.ones(2))),
        }
    }
]

In [ ]:
# fit the model with manually chosen hyperparameters
model = bp.models.GamModel('co2', gam_specs((np.log(0.001), np.log(0.1)))).fit(data, obs_cov=0.1**2)

In [ ]:
plt.figure(figsize=(8,6))

plt.subplot(211)
plt.plot(data.time, data.co2, 'r.', ms=3)
plt.plot(data.time, model.predict(data), 'k-', lw=1)
plt.grid(True)
plt.xlabel('time (months since 03/1958)')
plt.ylabel('co2')
plt.legend(['data', 'f1(time)+f2(month)'])

plt.subplot(223)
plt.plot(time_fit, model.theta['f(time)'], 'k-')
plt.grid(True)
plt.xlabel('time')
plt.ylabel('f1(time): smooth')

plt.subplot(224)
plt.plot(month_fit, model.theta['f(month)'], 'k-')
plt.grid(True)
plt.xlabel('month')
plt.ylabel('f2(month): smooth + periodic')

plt.tight_layout()
plt.show()

In [ ]:
# get the likelihood function for hyperparameter optimization
like = lambda v: bp.models.GamModel('co2', gam_specs(v)).fit(data, obs_cov=0.5**2).L

In [ ]:
# fit the model with optimized hyperparameters
v0=(np.log(0.001), np.log(0.1))
model_like = bp.models.GamModel('co2', gam_specs(minimize(like, v0, method='Nelder-Mead').x)).fit(data, obs_cov=0.5**2)

In [ ]:
plt.figure(figsize=(8,6))

plt.subplot(211)
plt.plot(data.time, data.co2, 'r.', ms=3)
plt.plot(data.time, model_like.predict(data), 'k-', lw=1)
plt.grid(True)
plt.xlabel('time (months since 03/1958)')
plt.ylabel('co2')
plt.legend(['data', 'f1(time)+f2(month)'])

plt.subplot(223)
plt.plot(time_fit, model_like.theta['f(time)'], 'k-')
plt.grid(True)
plt.xlabel('time')
plt.ylabel('f1(time): smooth')

plt.subplot(224)
plt.plot(month_fit, model_like.theta['f(month)'], 'k-')
plt.grid(True)
plt.xlabel('month')
plt.ylabel('f2(month): smooth + periodic')

plt.tight_layout()
plt.show()